# Python Decorators

**Master function enhancement patterns used in production systems**

---

## 🎯 The Problem

You're building a data transformation framework like Odibi:
- Users write transformation functions
- You need to **register** them in a global catalog
- You want **timing** and **logging** for all transformations
- You need **validation** of function signatures
- Functions should **explain themselves** (generate documentation)

**Without decorators:**
```python
# ❌ Manual registration - easy to forget!
def filter_records(df, threshold):
    return df[df.value > threshold]

registry.register('filter_records', filter_records)  # Boilerplate!
```

**With decorators:**
```python
# ✅ Automatic registration - clean and explicit
@transformation('filter_records')
def filter_records(df, threshold):
    return df[df.value > threshold]
```

---

## 🦉 First Principles: What is a Decorator?

### Core Concept

**A decorator is a function that takes a function and returns a function.**

```python
def decorator(func):         # Takes a function
    def wrapper(*args, **kwargs):  # Returns a new function
        # Do something before
        result = func(*args, **kwargs)
        # Do something after
        return result
    return wrapper
```

### The @ Syntax

```python
@decorator
def my_function():
    pass
```

Is equivalent to:

```python
def my_function():
    pass
my_function = decorator(my_function)
```

The @ syntax is **syntactic sugar** for wrapping functions.

---

## ⚡ Part 1: Basic Decorators

In [ ]:
import time
from functools import wraps

def timer(func):
    """Decorator that times function execution."""
    @wraps(func)  # Preserves original function metadata
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        print(f"⏱️ {func.__name__} took {elapsed:.4f}s")
        return result
    return wrapper

@timer
def slow_function():
    """Simulates a slow operation."""
    time.sleep(0.1)
    return "Done!"

result = slow_function()
print(result)

### Why @wraps?

Without `@wraps`, the decorated function loses its metadata:

In [ ]:
# Without @wraps
def bad_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

# With @wraps
from functools import wraps
def good_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@bad_decorator
def bad_func():
    """Original docstring."""
    pass

@good_decorator
def good_func():
    """Original docstring."""
    pass

print(f"Bad function name: {bad_func.__name__}")  # 'wrapper' ❌
print(f"Good function name: {good_func.__name__}")  # 'good_func' ✅

print(f"\nBad docstring: {bad_func.__doc__}")  # None ❌
print(f"Good docstring: {good_func.__doc__}")  # 'Original docstring.' ✅

---

## ⚡ Part 2: Decorators with Arguments

In [ ]:
def repeat(times):
    """Decorator factory that repeats function execution."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(times):
                print(f"Execution {i+1}/{times}")
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(times=3)
def greet(name):
    print(f"Hello, {name}!")

greet("Hassan")

### Understanding the Nesting

```python
@repeat(times=3)
def greet(name):
    pass
```

Expands to:

```python
greet = repeat(times=3)(greet)
#       ^^^^^^^^^^^^^^^ Returns decorator
#                      ^^^^^^^ Decorator wraps greet
```

- `repeat(times=3)` returns a decorator
- That decorator wraps `greet`

---

## ⚡ Part 3: Stacking Decorators

In [ ]:
def uppercase(func):
    """Decorator that uppercases result."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

def exclaim(func):
    """Decorator that adds exclamation."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"{result}!"
    return wrapper

@exclaim
@uppercase
def greet(name):
    return f"hello, {name}"

print(greet("hassan"))

### Execution Order

Decorators are applied **bottom to top**:

```python
@exclaim       # Applied SECOND
@uppercase     # Applied FIRST
def greet(name):
    return f"hello, {name}"
```

Equivalent to:
```python
greet = exclaim(uppercase(greet))
```

Flow:
1. `greet("hassan")` → `"hello, hassan"`
2. `uppercase` wrapper → `"HELLO, HASSAN"`
3. `exclaim` wrapper → `"HELLO, HASSAN!"`

---

## ⚡ Part 4: Class Decorators

In [ ]:
class CountCalls:
    """Decorator class that counts function calls."""
    
    def __init__(self, func):
        self.func = func
        self.count = 0
        # Preserve metadata
        self.__name__ = func.__name__
        self.__doc__ = func.__doc__
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"📞 Call #{self.count} to {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello(name):
    """Greet someone."""
    return f"Hello, {name}!"

print(say_hello("Alice"))
print(say_hello("Bob"))
print(say_hello("Charlie"))
print(f"\nTotal calls: {say_hello.count}")

### When to Use Class Decorators

Use class decorators when you need:
- **State** (like counting calls)
- **Multiple methods** on the decorator
- **Complex configuration**

Otherwise, function decorators are simpler.

---

## ⚡ Part 5: Registry Pattern (Like Odibi)

In [ ]:
from typing import Callable, Dict, Any
from functools import wraps

class TransformRegistry:
    """Global registry for transformations."""
    
    def __init__(self):
        self._transforms: Dict[str, Callable] = {}
        self._metadata: Dict[str, Dict[str, Any]] = {}
    
    def register(self, name: str, category: str = None):
        """Decorator to register a transformation."""
        def decorator(func: Callable) -> Callable:
            # Validate
            if name in self._transforms:
                raise ValueError(f"Transform '{name}' already registered")
            
            if not func.__doc__:
                raise ValueError(f"Transform '{name}' must have docstring")
            
            # Store
            self._transforms[name] = func
            self._metadata[name] = {
                'category': category,
                'docstring': func.__doc__
            }
            
            print(f"✅ Registered '{name}' [{category or 'uncategorized'}]")
            return func
        
        return decorator
    
    def get(self, name: str) -> Callable:
        """Get transformation by name."""
        if name not in self._transforms:
            raise ValueError(f"Transform '{name}' not found")
        return self._transforms[name]
    
    def list_all(self) -> Dict[str, Any]:
        """List all registered transformations."""
        return {
            name: {
                'function': func,
                **self._metadata[name]
            }
            for name, func in self._transforms.items()
        }

# Global registry instance
registry = TransformRegistry()

# Register transformations
@registry.register('filter_threshold', category='filtering')
def filter_threshold(df, threshold: float):
    """Filter records above threshold."""
    return df[df.value > threshold]

@registry.register('aggregate_sum', category='aggregation')
def aggregate_sum(df, group_by: str):
    """Sum values by group."""
    return df.groupby(group_by).sum()

# List all
print("\n📚 Registered Transformations:")
for name, info in registry.list_all().items():
    print(f"  - {name}: {info['docstring'].strip()} [{info['category']}]")

---

## ⚡ Part 6: Property Decorators

In [ ]:
class DataPipeline:
    """Example using property decorators."""
    
    def __init__(self, name: str):
        self._name = name
        self._status = "idle"
    
    @property
    def name(self) -> str:
        """Get pipeline name."""
        return self._name
    
    @property
    def status(self) -> str:
        """Get current status."""
        return self._status
    
    @status.setter
    def status(self, value: str):
        """Set status with validation."""
        allowed = ['idle', 'running', 'completed', 'failed']
        if value not in allowed:
            raise ValueError(f"Status must be one of {allowed}")
        print(f"Status: {self._status} → {value}")
        self._status = value

pipeline = DataPipeline("ETL Pipeline")
print(f"Name: {pipeline.name}")
print(f"Status: {pipeline.status}")

# Use setter
pipeline.status = "running"
pipeline.status = "completed"

# Try invalid value
try:
    pipeline.status = "invalid"
except ValueError as e:
    print(f"❌ Error: {e}")

---

## ⚡ Part 7: Context Managers (@contextmanager)

In [ ]:
from contextlib import contextmanager
import time

@contextmanager
def timer_context(name: str):
    """Context manager for timing code blocks."""
    print(f"⏱️ Starting {name}...")
    start = time.time()
    try:
        yield  # Code block executes here
    finally:
        elapsed = time.time() - start
        print(f"✅ {name} completed in {elapsed:.4f}s")

# Use it
with timer_context("Data Processing"):
    time.sleep(0.2)
    print("  Processing data...")

print("\nContext manager finished!")

---

## 🔍 Part 8: Analyzing Odibi's @transformation Decorator

Let's examine how Odibi's transformation decorator works!

In [ ]:
# Read Odibi's decorator implementation
decorator_path = r'c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi\odibi\transformations\decorators.py'

with open(decorator_path, 'r') as f:
    print(f.read())

### Key Patterns in Odibi's Decorator:

1. **Decorator Factory** - Takes arguments (name, version, category)
2. **Registry Pattern** - Registers function in global registry
3. **Wrapper Pattern** - Wraps function with `ExplanationDecorator`
4. **Validation** - Ensures docstrings exist
5. **Metadata** - Stores version, category, tags

See [odibi_transform_decorator.md](odibi_transform_decorator.md) for detailed analysis.

---

## 🏗️ Build: Complete Registry System

In [ ]:
from typing import Callable, Dict, Any, Optional
from functools import wraps
import inspect

class TransformationRegistry:
    """Production-quality transformation registry."""
    
    def __init__(self):
        self._transforms: Dict[str, Callable] = {}
        self._metadata: Dict[str, Dict[str, Any]] = {}
    
    def register(
        self,
        name: str,
        category: Optional[str] = None,
        version: str = "1.0.0",
        validate_signature: bool = True
    ):
        """Register transformation with metadata."""
        def decorator(func: Callable) -> Callable:
            # Validation
            if name in self._transforms:
                raise ValueError(f"Transform '{name}' already registered")
            
            if not func.__doc__ or len(func.__doc__.strip()) < 10:
                raise ValueError(
                    f"Transform '{name}' must have docstring (min 10 chars)"
                )
            
            # Validate signature
            if validate_signature:
                sig = inspect.signature(func)
                if len(sig.parameters) < 1:
                    raise ValueError(
                        f"Transform '{name}' must have at least 1 parameter"
                    )
            
            # Store
            self._transforms[name] = func
            self._metadata[name] = {
                'category': category,
                'version': version,
                'docstring': func.__doc__.strip(),
                'signature': str(inspect.signature(func))
            }
            
            return func
        
        return decorator
    
    def get(self, name: str) -> Callable:
        """Get transformation by name."""
        if name not in self._transforms:
            available = ', '.join(self._transforms.keys())
            raise ValueError(
                f"Transform '{name}' not found. Available: {available}"
            )
        return self._transforms[name]
    
    def list_by_category(self, category: str) -> Dict[str, Callable]:
        """List transformations by category."""
        return {
            name: func
            for name, func in self._transforms.items()
            if self._metadata[name]['category'] == category
        }
    
    def get_info(self, name: str) -> Dict[str, Any]:
        """Get full info for transformation."""
        return {
            'name': name,
            'function': self._transforms[name],
            **self._metadata[name]
        }

# Create registry
registry = TransformationRegistry()

# Register transformations
@registry.register('filter_nulls', category='filtering', version='1.0.0')
def filter_nulls(df, column: str):
    """Remove null values from specified column."""
    return df[df[column].notna()]

@registry.register('add_column', category='transformation', version='2.1.0')
def add_column(df, column: str, value: Any):
    """Add new column with constant value."""
    df[column] = value
    return df

# Test it
print("✅ Registered transformations:")
for name in registry._transforms:
    info = registry.get_info(name)
    print(f"\n{name} ({info['version']}) [{info['category']}]")
    print(f"  {info['docstring']}")
    print(f"  Signature: {info['signature']}")

---

## ✅ Testing Decorators

In [ ]:
# Test validation errors
test_registry = TransformationRegistry()

# Test: Missing docstring
try:
    @test_registry.register('bad_transform')
    def bad_transform(df):
        return df
except ValueError as e:
    print(f"✅ Caught missing docstring: {e}\n")

# Test: Duplicate registration
try:
    @test_registry.register('my_transform')
    def transform_1(df):
        """First transform."""
        return df
    
    @test_registry.register('my_transform')  # Duplicate!
    def transform_2(df):
        """Second transform."""
        return df
except ValueError as e:
    print(f"✅ Caught duplicate: {e}\n")

# Test: Invalid signature
try:
    @test_registry.register('no_params')
    def no_params():
        """No parameters!"""
        return None
except ValueError as e:
    print(f"✅ Caught invalid signature: {e}")

---

## 🎯 Summary

You now know:

✅ **Decorators** - Functions that wrap functions  
✅ **@wraps** - Preserve function metadata  
✅ **Arguments** - Decorator factories (nested functions)  
✅ **Stacking** - Multiple decorators (bottom-to-top)  
✅ **Class Decorators** - When you need state  
✅ **Registry Pattern** - Global registration like Odibi  
✅ **@property** - Getter/setter with validation  
✅ **@contextmanager** - Custom `with` blocks  

**Next Steps:**
1. Complete [exercises.ipynb](exercises.ipynb)
2. Read [odibi_transform_decorator.md](odibi_transform_decorator.md)
3. Study Odibi's transformation codebase

---

## 📚 Resources

- [PEP 318 - Decorators](https://peps.python.org/pep-0318/)
- [functools.wraps](https://docs.python.org/3/library/functools.html#functools.wraps)
- [contextlib](https://docs.python.org/3/library/contextlib.html)
- [Odibi Transformations](c:/Users/hodibi/OneDrive - Ingredion/Desktop/Repos/Odibi/odibi/transformations/)